In [2]:
# scripts/make_loo_untuned_bywindow_tables.py
# Build ONE LaTeX file with sections (one per window) for LOO UNTUNED results
# Output: results/_loo_untuned_bywindow.tex

import pandas as pd
import numpy as np
from pathlib import Path

root = Path("results")

def latex_escape(s: str) -> str:
    s = str(s)
    s = s.replace("\\", r"\textbackslash{}")
    s = s.replace("&", r"\&").replace("%", r"\%").replace("#", r"\#")
    s = s.replace("_", r"\_").replace("{", r"\{").replace("}", r"\}")
    return s

def parse_meta_from_filename(p: Path):
    """
    Expected filename tokens:
      <protocol>__<lag>__<zero>__<window>__<tuned/untuned>__summary__<ts>.csv
    """
    toks = p.stem.split("__")
    meta = {"protocol": None, "lag": None, "zero_policy": None,
            "window": None, "tuned": None, "run_tag": p.stem}
    if len(toks) >= 1:
        meta["protocol"] = toks[0]  # 'loo' expected here
    if len(toks) >= 2:
        meta["lag"] = toks[1]       # 'no_lag' | 'precip_lags' | 'both_lags'
    if len(toks) >= 3:
        meta["zero_policy"] = toks[2]  # 'standard' | 'train_keeps_zeros'
    if len(toks) >= 4:
        w = toks[3]  # 'YYYY-MM-DD_YYYY-MM-DD' or 'YYYY-MM-DD_to_YYYY-MM-DD'
        if "_to_" in w:
            ws, we = w.split("_to_", 1)
            meta["window"] = f"{ws}_to_{we}"
        elif "_" in w:
            ws, we = w.split("_", 1)
            meta["window"] = f"{ws}_to_{we}"
        else:
            meta["window"] = w
    if len(toks) >= 5:
        tt = toks[4].lower()
        meta["tuned"] = (tt == "tuned")
    return meta

def is_loo_untuned_filename(p: Path) -> bool:
    """Filter by filename: protocol must be 'loo', tuner token must be 'untuned'."""
    toks = p.stem.split("__")
    if len(toks) < 6:  # need at least ... __untuned__summary__...
        return False
    protocol = toks[0].lower()
    tuned_tok = toks[4].lower()
    return (protocol == "loo") and (tuned_tok == "untuned")

# collect LOO untuned files by filename meta
summary_files = [p for p in root.rglob("*__summary__*.csv") if is_loo_untuned_filename(p)]
if not summary_files:
    raise FileNotFoundError("No LOO untuned summary files found (by filename).")

rows = []
for p in summary_files:
    try:
        df = pd.read_csv(p)
        meta = parse_meta_from_filename(p)
        # attach enforced filename meta (override any inconsistent CSV columns)
        for k, v in meta.items():
            df[k] = v

        # normalize typical columns (robust to generator version differences)
        if "target" not in df.columns and "Target" in df.columns:
            df["target"] = df["Target"]

        if "MAE" not in df.columns and "mae" in df.columns:
            df["MAE"] = df["mae"]
        if "RMSE" not in df.columns and "rmse" in df.columns:
            df["RMSE"] = df["rmse"]

        # Known rows column
        if "n_known" not in df.columns:
            if "known_rows" in df.columns:
                df["n_known"] = df["known_rows"]
            elif "n_rows_known" in df.columns:
                df["n_known"] = df["n_rows_known"]
            else:
                df["n_known"] = np.nan

        # clean window text for display
        if "window" in df.columns:
            df["WindowTxt"] = (df["window"].astype(str)
                               .str.replace("_", " ", regex=False)
                               .str.replace("to", "–", regex=False))
        else:
            df["WindowTxt"] = "—"

        rows.append(df)
    except Exception as e:
        print(f"[skip] {p}: {e}")

if not rows:
    raise RuntimeError("Found files, but none could be parsed. Inspect CSV formats.")

df = pd.concat(rows, ignore_index=True)

# coerce types and drop rows missing metrics
df["MAE"]     = pd.to_numeric(df["MAE"], errors="coerce")
df["RMSE"]    = pd.to_numeric(df["RMSE"], errors="coerce")
df["n_known"] = pd.to_numeric(df["n_known"], errors="coerce").fillna(0).astype(int)
df = df.dropna(subset=["MAE","RMSE"]).copy()
if df.empty:
    raise RuntimeError("After coercion, no rows have valid MAE/RMSE. Check the CSV content.")

# compact labels
lag_map  = {"no_lag":"No-lag", "precip_lags":"Precip-lags", "both_lags":"Both-lags"}
zero_map = {"standard":"0→NaN train+test", "train_keeps_zeros":"train keeps 0; test 0→NaN"}

df["Lag"]         = df["lag"].map(lag_map).fillna(df["lag"].astype(str))
df["Zero policy"] = df["zero_policy"].map(zero_map).fillna(df["zero_policy"].astype(str))

# unique windows present
windows = list(df["WindowTxt"].dropna().unique())
windows.sort()

out_tex = root / "_loo_untuned_bywindow.tex"
with open(out_tex, "w", encoding="utf-8") as f:
    f.write("\\begin{table}[htbp]\n\\centering\n\\small\n")
    f.write("\\setlength{\\tabcolsep}{4pt}\n\\renewcommand{\\arraystretch}{1.12}\n")
    f.write("\\caption{LOO (untuned) results, presented separately by analysis window. "
            "Metrics computed on originally known test rows only.}\n")
    f.write("\\label{tab:loo-untuned-bywindow}\n")

    for wi, wtxt in enumerate(windows):
        block = df[df["WindowTxt"] == wtxt].copy()
        f.write(f"\\subsubsection*{{Window: {latex_escape(wtxt)}}}\n")
        f.write("\\begin{tabularx}{\\textwidth}{l l X r r r}\n\\toprule\n")
        f.write("Target & Lag & Zero policy & $n_{\\mathrm{known}}$ & MAE & RMSE\\\\\n\\midrule\n")

        if block.empty:
            f.write("\\multicolumn{6}{c}{No runs found for this window}\\\\\n")
        else:
            block = (block[["target","Lag","Zero policy","n_known","MAE","RMSE"]]
                     .rename(columns={"target":"Target"}))
            block = block.sort_values(["Target","RMSE","MAE"]).reset_index(drop=True)
            for _, r in block.iterrows():
                f.write(
                    f"{latex_escape(r['Target'])} & "
                    f"{latex_escape(r['Lag'])} & "
                    f"{latex_escape(r['Zero policy'])} & "
                    f"{int(r['n_known'])} & "
                    f"{r['MAE']:.3f} & {r['RMSE']:.3f}\\\\\n"
                )

        f.write("\\bottomrule\n\\end{tabularx}\n")
        if wi < len(windows) - 1:
            f.write("\\vspace{0.75em}\n")

    f.write("\\end{table}\n")

print(f"[OK] wrote {out_tex}")


[OK] wrote results/_loo_untuned_bywindow.tex
